In [1]:
import pandas as pa
import numpy as np
import random
import csv
import math
with open("D:/Download/iris.csv", newline= "", encoding="ISO-8859-1") as filecsv:  #importo i dati dal file csv
    DATI = pa.read_csv('D:/Download/iris.csv') 
    XComplete = DATI.iloc[:,0:4] 
    yComplete = DATI.iloc[:,4]
#print(XComplete)
#print(yComplete)
setSize = np.size(yComplete)
l = list(range(setSize))
random.shuffle(l)
XComplete = XComplete.iloc[l,:]
yComplete = yComplete.iloc[l]
size_train = math.ceil(setSize * 0.6)
size_cv = math.ceil(setSize * 0.2)
X = XComplete.iloc[0:size_train,:]
X_Numpy = X.to_numpy(float)
y = yComplete.iloc[0:90]
y_Numpy = y.to_numpy(float)
X_cv = XComplete.iloc[size_train : size_train+size_cv,:]
X_cv_Numpy = X_cv.to_numpy(float)
y_cv = yComplete.iloc[size_train : size_train+size_cv]
y_cv_Numpy = y_cv.to_numpy(float)
x_test = XComplete.iloc[size_train+size_cv:setSize,:]
X_test_Numpy = x_test.to_numpy(float)
y_test = yComplete.iloc[size_train+size_cv:setSize]

def featureNormalize(X):
    X_norm = X
    mu = X.mean(0)
    sigma = X.std(0)                                                      #implemento la feature normalize
    r,c = X.shape
    for i in range(c):
        X_norm[:,i] = ((X[:,i] - mu[i]) / sigma[i])
    return X_norm

X_norm = featureNormalize(X_Numpy)
X_cv_norm = featureNormalize(X_cv_Numpy)
X_test_norm = featureNormalize(X_test_Numpy)

dimensioneLayer = [4, 1, 1]                                             #Specifico le dimensioni della rete neurale

def sigmoide(Z):
    s = 1 / (1 + np.exp(-1 * Z))                                              #Implemento la sigmoide
    return s

def sigmoide_backward(dA, s):
    gC = dA * s * (1-s)                                             #in questa funzione dA rappresenta il risultato
    return gC                                                       #dell'attivazione, mentre gC rappresenta il gradiente                                                     

def inizializzaParametri(dimensioneLayer):
    np.random.seed(1)
    parametri = {}
    L = len(dimensioneLayer)                                                 #Inizializzo i parameti, dove W sono i pesi
    for l in range(1, L):                                                    #e b rappresentano le bias unit
        parametri['W' + str(l)] = np.random.randn(dimensioneLayer[l], dimensioneLayer[l-1]) / np.sqrt(dimensioneLayer[l-1])
        parametri['b' + str(l)] = np.zeros((dimensioneLayer[l], 1))
    return parametri

parametri = inizializzaParametri(dimensioneLayer)
#print("I paramtri inizializzati sono: ",parametri)

def attivazioneForward(aPrec, W, b):
    Z = np.dot(W,aPrec)+b                                   #aPrec rappresenta l'attivazione del livello precedente
    A = sigmoide(Z)
    return A

def modelloForward(X,parameters):
    A=X.transpose()                                        #implemento la forward propagation
    L=len(parameters)//2                                   #Al rappresenta il valore finale dell'attivazione
    for l in range(1,L):
        aPrec=A
        A = attivazioneForward(aPrec, parametri["W"+str(l)], parametri["b"+str(l)])        
    AL = attivazioneForward(A,parametri["W"+str(L)],parametri["b"+str(L)])
    return AL, A, aPrec

AL, sigmoide, aPrec = modelloForward(X_norm, parametri)

def attivazioneBackward(dA):
    dZ = sigmoide_backward(dA, sigmoide)                   
    m = aPrec.shape[1]
    L=len(parametri)//2                                    #dW e db rappresentano, rispettivamete, i gradienti rispetto 
    for l in range(1,L):                                   #ai pesi e alle bias
        dW = 1./m * np.dot(dZ,aPrec.T)
        db = 1./m * np.sum(dZ, axis = 1)
    return  dW, db

def modelloBackward(AL, Y):
    grads = {}
    L = len(dimensioneLayer) 
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)                                    #implemento la backward propagation
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    for l in range(1,L):
        grads["dW" + str(l)], grads["db" + str(l)] = attivazioneBackward(dAL)
    return grads

grads = modelloBackward(AL, y_Numpy)
#print(grads)

learning_rate = 0.01
L=len(parametri)//2
for l in range(L):                                   #Aggiorno i parametri facendo riferimeto alla backPropagation
    parametri["W"+str(l+1)]=parametri["W"+str(l+1)]-learning_rate*grads["dW"+str(l+1)]
    parametri["b"+str(l+1)]=parametri["b"+str(l+1)]-learning_rate*grads["db"+str(l+1)]
print(parametri)



{'W1': array([[ 0.81840118, -0.30908773, -0.25648911, -0.52884857]]), 'b1': array([[0.01349095]]), 'W2': array([[0.87163612, 0.8621981 , 0.8730044 , 0.87304337]]), 'b2': array([[0.01349095]])}
